In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Missing').getOrCreate()

In [2]:
## Read The dataset
training = spark.read.csv('test1.csv',header=True,inferSchema=True)

In [3]:
training.show()

+--------+---+----------+------+
|    Name|age|Experience|Salary|
+--------+---+----------+------+
|Debarjun| 21|         3| 30000|
|   Arjun| 22|         2| 25000|
|   Karan| 25|         1| 20000|
|      Om| 20|         2| 20000|
|   Omkar| 19|         1| 15000|
|   Tanay| 18|         2| 18000|
+--------+---+----------+------+



In [4]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [5]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [6]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["age","Experience"],outputCol="Independent Features")

In [7]:
output=featureassembler.transform(training)

In [8]:
output.show()

+--------+---+----------+------+--------------------+
|    Name|age|Experience|Salary|Independent Features|
+--------+---+----------+------+--------------------+
|Debarjun| 21|         3| 30000|          [21.0,3.0]|
|   Arjun| 22|         2| 25000|          [22.0,2.0]|
|   Karan| 25|         1| 20000|          [25.0,1.0]|
|      Om| 20|         2| 20000|          [20.0,2.0]|
|   Omkar| 19|         1| 15000|          [19.0,1.0]|
|   Tanay| 18|         2| 18000|          [18.0,2.0]|
+--------+---+----------+------+--------------------+



In [9]:
finalized_data=output.select("Independent Features","Salary")

In [10]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|          [21.0,3.0]| 30000|
|          [22.0,2.0]| 25000|
|          [25.0,1.0]| 20000|
|          [20.0,2.0]| 20000|
|          [19.0,1.0]| 15000|
|          [18.0,2.0]| 18000|
+--------------------+------+



In [11]:
from pyspark.ml.regression import LinearRegression
##train test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor=regressor.fit(train_data)

In [12]:
### Coefficients
regressor.coefficients

DenseVector([1510.6383, 8021.2766])

In [13]:
### Intercepts
regressor.intercept

-25468.085106383038

In [14]:
### Prediction
pred_results=regressor.evaluate(test_data)

In [16]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [19.0,1.0]| 15000|11255.319148936123|
+--------------------+------+------------------+



In [17]:

pred_results.meanAbsoluteError,pred_results.meanSquaredError

(3744.6808510638766, 14022634.67632448)